## Dice Game - Parameter Optimisation Using WandB
### This notebook is for reference of parameter tuning only

In [1]:
%pip install wandb

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dice_game import DiceGame
import numpy as np

In [3]:
from abc import ABC, abstractmethod

class DiceGameAgent(ABC):
    def __init__(self, game):
        self.game = game
    
    @abstractmethod
    def play(self, state):
        pass

def play_game_with_agent(agent, game, verbose=False):
    state = game.reset()
    
    if(verbose): print(f"Testing agent: \n\t{type(agent).__name__}")
    if(verbose): print(f"Starting dice: \n\t{state}\n")
    
    game_over = False
    actions = 0
    while not game_over:
        action = agent.play(state)
        actions += 1
        
        if(verbose): print(f"Action {actions}: \t{action}")
        _, state, game_over = game.roll(action)
        if(verbose and not game_over): print(f"Dice: \t\t{state}")

    if(verbose): print(f"\nFinal dice: {state}, score: {game.score}")
        
    return game.score

In [4]:

class MyAgent(DiceGameAgent):
    def __init__(self, game, theta=0.2, gamma=1):
        super().__init__(game)
        self.gamma = gamma
        self.theta = theta
        self.thSquared = theta**2
        self.values = self.valueIteration()
    
    
    #MDP implementation. Calculate action value (expected)
    def calcActionValue(self, action, state):
        states, end, reward, prob = self.nextStates[state][action]
        if end:
            return self.game.final_scores[state]
        expected = sum([self.valueAction[s] * p for p, s in zip(prob, states)])
        return reward + self.gamma * expected

    #Get next states from game object
    def getNextStates(self,key,state):
        return self.game.get_next_states(key,state)
    
    #Calculate the value of a state
    def stateValue(self, state, prob, dict,end,reward):
        if end:
            return reward * prob
        return (reward + self.gamma * dict[state][0]) * prob
        

    #One step look ahead for all actions
    def nextActionsList(self,dictionary,state):
        ActionDict = {a:0 for a in self.game.actions}
        for actionKey in ActionDict.keys():
            nextStates, end, reward, prob = self.getNextStates(actionKey,state)
            #Loop through all the next states
            for nextState, prob in zip(nextStates, prob):
                ActionDict[actionKey] += self.stateValue(nextState, prob, dictionary, end, reward)
        return ActionDict

    def valueIteration(self):
        valuesDictionary = {s: [0, None] for s in self.game.states}
        while True:
            delta = 0
            #Loop through all states
            for state in self.game.states:
                actionList = self.nextActionsList(valuesDictionary,state)
                #Get best action
                bestActionValue = max(actionList.values())
                #Change to either previous delta or max difference
                delta = max(delta, abs(valuesDictionary[state][0] - bestActionValue))
                delSquared = delta ** 2
                #Update value
                valuesDictionary[state] = [bestActionValue, max(actionList, key=actionList.get)]
            if delSquared < self.thSquared:
                return valuesDictionary

        
    def play(self, state):
        return self.values[state][1]

In [5]:
# Import and log in to wandb
import wandb
!wandb login # You will be prompted to paste an API key here

wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Beast/.netrc
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: michalis. Use `wandb login --relogin` to force relogin


In [6]:
# Set config dict - parameters to iterate over, search method, and objective
sweep_config = {
  "method": "grid",
  "metric": {"name": "Avg Score", "goal": "maximize"},
  "parameters": {
        "gamma": {
                "values": [0.955, 0.96, 0.965, 0.97, 0.975, 0.98, 0.985, 0.99, 0.995, 1]},
      "theta": {
            "values": [0.1, 0.01, 0.001, 0.05, 0.005]
        }
    }
}

In [7]:
import time

def train():
    """
    The core train function that the wandb program will run iteratively for each parameter in the sweep config
    """
    wandb.init()
    config = wandb.config
    np.random.seed(10)
    n = 10000

    game = DiceGame()
    total_score = 0
    total_time = 0
    start_time = time.process_time()
    test_agent = MyAgent(game, config.theta, config.gamma)
    total_time += time.process_time() - start_time
    for i in range(n):
        start_time = time.process_time()
        score = play_game_with_agent(test_agent, game, verbose=False)
        total_time += time.process_time() - start_time
        total_score += score
    wandb.log({"Theta": config.theta, "Gamma": config.gamma, "Avg Score": total_score/n, "Avg Time": total_time/n})
    print(f"Average score: {total_score/n}")

In [8]:
# A sweep is the name of the iterative parameter selection process
sweep_id = wandb.sweep(sweep_config, project='DiceGameNew')

# Run the sweep
wandb.agent(sweep_id, function=train)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: dw19vts6
Sweep URL: https://wandb.ai/michalis/DiceGameNew/sweeps/dw19vts6


wandb: Agent Starting Run: tcn24amj with config:
wandb: 	gamma: 0.955
wandb: 	theta: 0.1


Average score: 13.0183


Exception in thread ChkStopThr:
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.2032.0_x64__qbz5n2kfra8p0\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.2032.0_x64__qbz5n2kfra8p0\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Beast\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\wandb\sdk\wandb_run.py", line 190, in check_status
    status_response = self._interface.communicate_stop_status()
  File "C:\Users\Beast\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\wandb\sdk\interface\interface.py", line 128, in communicate_stop_status
    resp = self._communicate_stop_status(status)
  File "C:\Users\Beast\AppData\Local\Packages\

...

Average score: 13.3677


Avg Score,▁
Avg Time,▁
Gamma,▁
Theta,▁
Avg Score,13.3677
Avg Time,0.00017
Gamma,1
Theta,0.005


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


Error in callback <function _WandbInit._pause_backend at 0x0000023B82CD2290> (for post_run_cell):


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host